In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.pardir)))
import lightgbm as LGB

In [3]:
import pandas as pd
import numpy  as np
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from src.paths import TRANSFORMED_DATA_DIR
from src.data_split import train_test_split

In [4]:
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tarbular_data.parquet')
df.head()

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hours,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0


In [5]:
X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022,6,1,0,0,0),
    target_column_name='target_rides_next_hour'
)
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32226, 674)
y_train.shape=(32226,)
X_test.shape=(56068, 674)
y_test.shape=(56068,)


In [8]:
import numpy as np
import optuna
from optuna.visualization import plot_intermediate_values
from src.model import get_pipeline
from sklearn.model_selection import KFold, TimeSeriesSplit

In [15]:
def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    ## Pick the hyper-parameters
    hyperparams = {
        "metric":"mae",
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100)
    }

    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    for train_index, val_index in tss.split(X_train):

        ## Split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], X_train.iloc[val_index]

        ## Train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)

        ## Evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)

    ## Return the mean mae
    print(scores)
    return np.array(scores).mean()
    

In [16]:
## Now we create a study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2025-05-09 20:11:04,652] A new study created in memory with name: no-name-ba307ec2-17ac-4b2e-8f18-4fa502774e92
[W 2025-05-09 20:11:06,752] Trial 0 failed with parameters: {'num_leaves': 62, 'feature_fraction': 0.29867927019344037, 'bagging_fraction': 0.8544117170419094, 'min_child_samples': 29} because of the following error: DTypePromotionError("The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float32DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DTyp

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float32DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DT